# Step 4 - Create array of unique Committees

In [1]:
import re
import os
import numpy as np
import pandas as pd
import ujson as json
from tqdm import tqdm
import sqlCommands

from dateutil.parser import parse as DateParser



def findOccurences(s, ch):
    res = list()
    for i in re.finditer(ch,s):
        res.append(i.start())
    return res

In [3]:
#set up engine
dbname = 'legislatr'
engine = sqlCommands.get_engine(dbname)
#query = "SELECT * FROM topics;"
bills_df = pd.read_sql_table('allbills4',engine)

committee = list(bills_df['committees'])
print(type(committee))

<class 'list'>


In [4]:
#find separate comittees and separate them.
new_comms = list()
for s in committee:
    #first check for empty
    if s == "{}":
        continue
    s= s.replace("{","")
    ind = findOccurences(s,",")
    #loop over each new sponsor
    sind = 0
    ind.append(-1)
    for i in ind:
        sub_comm = s[sind:i]
        sind = i+1
        sub_comm = sub_comm.replace("}","")
        new_comms.append(sub_comm)

In [5]:
u_new_comms = np.unique(new_comms)
print(len(u_new_comms)) #should be 865 based on playing before
print(u_new_comms[-10:-1]) #too may print lines

41
['SSEG' 'SSEV' 'SSFI' 'SSFR' 'SSGA' 'SSHR' 'SSJU' 'SSRA' 'SSSB']


In [7]:
#save unique committees list to database
comm_list_df = pd.DataFrame()
comm_list_df['committee'] = u_new_comms
sqlCommands.write_to_database(dbname,'committee_list',comm_list_df,engine)